# Imports

In [ ]:
import torch
import torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, random
import cv2 as cv
from matplotlib import pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode

from prepare_models import Models

import os

In [ ]:
print('Is CUDA available:',torch.cuda.is_available())
print('CUDA version:',torch.version.cuda)
print('Torch.version:',torch.__version__)
print('Detectron2 version:',detectron2.__version__)

In [ ]:
# prepare segmentation model
predictor = Models().preapre_segmentation_model()

In [ ]:
# prepare files
PATH = r"Z:\labeled_data_060622"
image_path_list = list()
for filename in os.listdir(PATH):
    if ".png" in filename: image_path_list.append(os.path.join(PATH,filename))

print(f"Loaded {len(image_path_list)} images!")

In [ ]:
import time

array_time_prediction = np.empty((len(image_path_list)))

for i,ImgPath in enumerate(image_path_list):
    im = cv.imread(ImgPath)
    time_start = time.time_ns()
    predictions = predictor(im)
    time_prediction = (time.time_ns()-time_start)/1e6 # miliseconds
    array_time_prediction[i] = time_prediction
    print(f"{i+1}/{len(image_path_list)}, time: {time_prediction}")

In [ ]:
mean_time = array_time_prediction.mean()
std_time = array_time_prediction.std()
max_time = (np.max(array_time_prediction),np.argmax(array_time_prediction))
min_time = (np.min(array_time_prediction),np.argmin(array_time_prediction))

if torch.cuda.is_available(): 
    device = torch.cuda.get_device_name(torch.cuda.current_device())
else:
    device = "CPU"

print(f'''
Device used: {device};\n
Mean time of prediciton: {mean_time:.3f}+-{2*std_time:.3f} ms;\n
Max time: {max_time[0]:.3f} ms for img: {image_path_list[max_time[1]]};\n
Min time: {min_time[0]:.3f} ms for img: {image_path_list[min_time[1]]};\n
''')
fig,axes = plt.subplots(ncols=2)
axes[0].imshow(cv.imread(image_path_list[max_time[1]]))
axes[0].set_title("Max time")
axes[1].imshow(cv.imread(image_path_list[min_time[1]]))
axes[1].set_title("Min time")
fig.show()
fig,ax = plt.subplots(ncols=1)
ax.plot(
    array_time_prediction, 
    linestyle="none", marker="o", label="Prediction time", color="black"
)
ax.axhline(max_time[0], label="MAX", color="red", linestyle="--", linewidth=1)
ax.axhline(min_time[0], label="MIN", color="blue", linestyle="--", linewidth=1)
ax.axhline(mean_time, label="MEAN", color="black", linestyle="-", linewidth=1)
ax.axvline(max_time[1], color="red", linestyle="--", linewidth=1)
ax.axvline(min_time[1], color="blue", linestyle="--", linewidth=1)
ax.grid(True)
ax.legend(loc="upper left")
ax.set_xlabel("Image")
ax.set_ylabel("Prediction time")
fig.show()